# 코사인유사도를 이용한 영화 추천 시스템

## 코사인 유사도

### numpy로 직접 계산

In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A,B):
    return dot(A, B)/(norm(A)*norm(B))

In [2]:
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

In [3]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


### sklearn 사용해서 계산

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
cosine_similarity([doc1],[doc2])

array([[0.66666667]])

## 유사도를 이용한 추천시스템

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [7]:
data = pd.read_csv('data/movies_metadata.csv', low_memory=False)

In [8]:
data.shape

(45466, 24)

In [9]:
df = data[['title','overview']]

In [10]:
df.overview[0] #이 오버뷰의 코사인 유사도를 가지고 온다
df = df.head(20000)

## 데이터 전처리

In [11]:
df.isnull().sum()

title         2
overview    135
dtype: int64

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
df.dropna(how='any', inplace=True) # how = 'any'가 디폴트

In [14]:
df.shape

(19863, 2)

In [15]:
df.tail() ## index 맞춰주어야 한다

,title,overview
19995,Rebellion,Dissidents in a French colony attack a police ...
19996,Versailles,A young mother Nina and her son Enzo find them...
19997,Two in the Wave,An in-depth analysis of the relationship betwe...
19998,Lotte Reiniger: Homage to the Inventor of the ...,Follows the life and work of animator Lotte Re...
19999,"RKO Production 601: The Making of 'Kong, the E...","An in-depth look at the genesis, production, a..."


In [16]:
df.set_index('title',inplace=True)
df.reset_index(inplace=True)
df.tail()

,title,overview
19858,Rebellion,Dissidents in a French colony attack a police ...
19859,Versailles,A young mother Nina and her son Enzo find them...
19860,Two in the Wave,An in-depth analysis of the relationship betwe...
19861,Lotte Reiniger: Homage to the Inventor of the ...,Follows the life and work of animator Lotte Re...
19862,"RKO Production 601: The Making of 'Kong, the E...","An in-depth look at the genesis, production, a..."


## 텍스트 전처리

In [17]:
import re
df['clean_doc'] = df.overview.apply(lambda x: re.sub('[^A-Za-z ]', '', x))#구두점만 제거
df['clean_doc'] = df.overview.str.replace('[^A-Za-z ]', '')#구두점만 제거
df.head()

,title,overview,clean_doc
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Led by Woody Andys toys live happily in his ro...
1,Jumanji,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Cheated on mistreated and stepped on the women...
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just when George Banks has recovered from his ...


## DTM 변환

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(stop_words='english')
tfidf_matrix = tvect.fit_transform(df.overview)
tfidf_matrix.shape

(19863, 47487)

In [19]:
tfidf_clean = tvect.fit_transform(df.clean_doc)
tfidf_clean.shape

(19863, 54245)

## 영화의 타이틀과 인덱스를 가진 테이블

In [20]:
indices = pd.Series(df.index, index=df.title).drop_duplicates()
indices.head() #숫자형태로 바꾸었다 일부러

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [21]:
indices.shape

(19863,)

## 코사인 유사도를 통해 유사한 영화를 찾는 함수

In [22]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#cosine_clean = linear_kernel(tfidf_clean, tfidf_clean)

In [23]:
cosine_sim.shape

(19863, 19863)

In [24]:
cosine_sim[1,:5] #미리 코사인 유사도를 구해놓고 함수에 쳐넣는다

array([0.01575156, 1.        , 0.04906868, 0.        , 0.        ])

In [25]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return df['title'].iloc[movie_indices]

In [26]:
get_recommendations('The Dark Knight Rises')

12447                            The Dark Knight
149                               Batman Forever
1314                              Batman Returns
15444                 Batman: Under the Red Hood
583                                       Batman
9203          Batman Beyond: Return of the Joker
17930                           Batman: Year One
19661    Batman: The Dark Knight Returns, Part 1
3077                Batman: Mask of the Phantasm
10092                              Batman Begins
Name: title, dtype: object